In [1]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [4]:
dataset = pd.read_csv('C:/Users/LAIBA/Downloads/Coursera.csv/Coursera.csv')
dataset.head()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         3522 non-null   object
 1   University          3522 non-null   object
 2   Difficulty Level    3522 non-null   object
 3   Course Rating       3522 non-null   object
 4   Course URL          3522 non-null   object
 5   Course Description  3522 non-null   object
 6   Skills              3522 non-null   object
dtypes: object(7)
memory usage: 192.7+ KB


In [6]:
tfidf = TfidfVectorizer(stop_words='english')
dataset['Course Description'] = dataset['Course Description'].fillna('')
#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
overview_matrix = tfidf.fit_transform(dataset['Course Description'])
#Output the shape of tfidf_matrix
overview_matrix.shape


(3522, 20074)

In [7]:
similarity_matrix = linear_kernel(overview_matrix,overview_matrix)
similarity_matrix

array([[1.00000000e+00, 3.12366523e-02, 1.97603991e-02, ...,
        3.17538002e-02, 3.33859933e-02, 1.96231367e-02],
       [3.12366523e-02, 1.00000000e+00, 8.58915185e-03, ...,
        3.13671991e-02, 4.88239107e-03, 4.56033552e-02],
       [1.97603991e-02, 8.58915185e-03, 1.00000000e+00, ...,
        3.45669421e-03, 1.65197252e-02, 6.37237740e-03],
       ...,
       [3.17538002e-02, 3.13671991e-02, 3.45669421e-03, ...,
        1.00000000e+00, 5.07544593e-04, 6.72367274e-03],
       [3.33859933e-02, 4.88239107e-03, 1.65197252e-02, ...,
        5.07544593e-04, 1.00000000e+00, 1.14068789e-03],
       [1.96231367e-02, 4.56033552e-02, 6.37237740e-03, ...,
        6.72367274e-03, 1.14068789e-03, 1.00000000e+00]])

In [8]:
#index mapping
mapping = pd.Series(dataset.index,index = dataset['Course Name'])
mapping

Course Name
Write A Feature Length Screenplay For Film Or Television                 0
Business Strategy: Business Model Canvas Analysis with Miro              1
Silicon Thin Film Solar Cells                                            2
Finance for Managers                                                     3
Retrieve Data using Single-Table SQL Queries                             4
                                                                      ... 
Capstone: Retrieving, Processing, and Visualizing Data with Python    3517
Patrick Henry: Forgotten Founder                                      3518
Business intelligence and data analytics: Generate insights           3519
Rigid Body Dynamics                                                   3520
Architecting with Google Kubernetes Engine: Production                3521
Length: 3522, dtype: int64

In [9]:
def recommend_course(course_name):
        Cname_index = mapping[course_name]
        #get similarity values with other courses
        #similarity_score is the list of index and similarity matrix
        similarity_score = list(enumerate(similarity_matrix[Cname_index]))
        #sort in descending order the similarity score of course name inputted with all the other courses
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        # Get the scores of the 10 most similar courses. Ignore the first course.
        similarity_score = similarity_score[1:10]
        #return course names using the mapping series
        course_indices = [i[0] for i in similarity_score]
        return (dataset['Course Name'].iloc[course_indices])

In [10]:
recommend_course('Finance for Managers')

1839    Fundamentals of financial and management accou...
1891          Accounting and Finance for IT professionals
1985                  Introduction to Finance: The Basics
419                    Finance for Non-Financial Managers
1164                         Corporate Finance Essentials
708     Understanding Financial Statements: Company Po...
1090                    Financial Accounting Fundamentals
590                Corporate finance: Know your numbers 2
3119    Introduction to Finance: The Role of Financial...
Name: Course Name, dtype: object

In [11]:
recommend_course('Corporate Finance Essentials')

3119    Introduction to Finance: The Role of Financial...
1985                  Introduction to Finance: The Basics
419                    Finance for Non-Financial Managers
646               Finance for Non-Financial Professionals
2256    Principles of Corporate Finance �  A Tale of V...
2082                Finance for Non-Finance Professionals
1891          Accounting and Finance for IT professionals
2829       Accounting: Principles of Financial Accounting
1034                      Understanding Financial Markets
Name: Course Name, dtype: object